In [1]:
# Importación de paquetes
import numpy as np # Cálculo numérico
import pandas as pd # Análisis de datos

### Parámetros del Algoritmo

In [2]:
# Tamaño de la población
numero_individuos = 20

# Número de corridas
total_epocas = 10

# Probabilidad de mutación
m = 0.45

### Función Fitness

In [3]:
# función de Ackley 2
def fitness (fenotipo):
    d = len(fenotipo) # número de dimensiones
    sum1 = 0 # suma dentro de la raíz de la función exp
    for i in range(0, d - 1): # cálculo de la sumatoria
        sum1 += fenotipo[i]**2
    return -200 * np.exp(-0.02 * sum1)

### Funciones de Codificación Genética

In [4]:
def decodificar (genotipo):
    fenotipo = np.zeros(3) # Inicializar variable
    coordenada = 2 # Coordenada actual = variable 3
    
    # Creamos un conjunto de índices que van desde 59 hasta 0 para representar la posición de los genes en el cromosoma
    posiciones = np.arange(len(genotipo) - 1, -1, -1)

    # Expresión decimal de la cadena binaria que representa una variable dentro del cromosoma
    decimal = 0.0

    # Valor real decodificado
    x = 0.0

    for posicion in posiciones:
        # Si el resto de la división por 20 es mayor que cero, estamos en un gen dentro de la misma variable.
        exponente = posicion % 20
        if exponente > 0:
            decimal += genotipo[posicion] * 2**(exponente)
            continue

        # Cuando llegue a cero, terminamos de decodificar la variable y cambiamos de coordenada
        decimal += genotipo[posicion]
        x = -32 + decimal * (64 / (2**20 - 1))
        fenotipo[coordenada] = np.round(x, 4)
        coordenada -= 1
        decimal = 0.0
    
    return fenotipo

# Función auxiliar para verificar que un individuo representa una solución factible
def validar (cromosoma):
    factible = True
    fenotipo = decodificar(cromosoma)

    # Para marcar un cromosoma como no factible, basta con que una de las coordenadas caiga fuera del rango [-32, 32]
    for i in range(0, len(fenotipo)):
        if fenotipo[i] > 32 or fenotipo[i] < -32:
            factible = False
            break

    return factible

# Función auxiliar para visualizar un cromosoma como una cadena de caracteres
def cromosoma_to_string (cromosoma):
    cadena = ''
    for gen in cromosoma:
        cadena = f'{cadena}{gen}'
    return cadena

### Función para Generar un Individuo Factible

In [5]:
def generar_individuo ():
    # Creamos un cromosoma genérico de 60 genes todos con alelo = 0
    cromosoma = np.zeros(60, int)

    # El objetivo es ir añadiendo 1's de menor a mayor posición según una ley Uniforme(0, 1) a cada coordenada hasta que se supere el máximo valor permitido: 32. Esto para que las soluciones que se generen de forma aleatoria estén dentro del dominio

    posiciones = np.arange(59, -1, -1)

    for posicion in posiciones:
        u = np.random.random()
        alelo = 1 if u < 0.5 else 0 # 50% de probabilidad
        
        # Si el alelo obtenido aleatoriamente es 0, no hay que hacer nada ya que la posición del cromosoma se encuentra en 0 por defecto. Si no, asignamos a la posición el alelo 1 y validamos que sea un cromosoma válido, si no es válido, dejamos el gen en 0
        if alelo == 1:
            cromosoma[posicion] = 1
            if not validar(cromosoma):
                cromosoma[posicion] = 0

    return cromosoma

### Funciones de Cruce y Mutación

La recombinación se hará copiando los genes del primer padre desde el gen 0 al gen 29 para luego copiar los genes del segundo padre desde el gen 30 hasta el gen 59 (mitad de los genes del padre 1 y mitad de los genes del padre 2)

In [6]:
def cruzar (padre1, padre2):
    nuevo_individuo = np.zeros(60, dtype=int) # Creamos una cadena binaria genérica
    for gen in range(0, 60):
        alelo = padre1[gen] if gen < 29 else padre2[gen]
        nuevo_individuo[gen] = alelo
    return nuevo_individuo

La mutación consistirá simplemente en invertir el alelo de cada gen:

In [7]:
def mutar (individuo):
    individuo_mutado = np.zeros(len(individuo), dtype=int)
    for gen in range(0, len(individuo)):
        individuo_mutado[gen] = 1 if individuo[gen] == 0 else 0
    return individuo_mutado

### Generación de la población inicial

In [8]:
# Inicialización de la tabla para registrar y mostrar la población
poblacion = []

# Añadimos a cada fila un individuo junto con su valor de fitness
for i in range(0, numero_individuos):
    genotipo = generar_individuo()
    fenotipo = decodificar(genotipo)
    poblacion.append({
        'Individuo' : cromosoma_to_string(genotipo),
        'Fenotipo' : fenotipo,
        'Fitness': fitness(fenotipo)
    })

# Creación del dataframe para visualizar la población
df_poblacion = pd.DataFrame(poblacion)

# Visualización de la población
poblacion_inicial = df_poblacion.style.set_properties(**{
    'font-size' : '12pt'
})
poblacion_inicial

,Individuo,Fenotipo,Fitness
0,000000001000100011110010110010110111011110000110010101010111,[28.2657 27.7064 26.6622],-0.000000
1,111100110101110101101011010111101100001100000011001001101101,[-5.0811 16.87 13.5743],-0.402535
2,110010110110100001100110111010000010111101011001101110101111,[-7.6433 29.0229 29.4626],-0.000003
3,010111101010111011111101100100101111111101110010100110011101,[29.8356 31.822 14.3954],-0.000000
4,101000001011000000010111101000101011111100001000111101010110,[ 0.2035 31.3183 -5.2646],-0.000001
5,011011111100000000010000101111001110001110111100100001011101,[ 0.0619 17.8096 14.5194],-0.351552
6,111010111111000001101000010111110100101010100000101010000010,[ -7.7525 -11.2558 -15.6716],-4.770407
7,101111010100110101000111101010001010110110001100001000001111,[-21.2072 13.2714 28.0655],-0.000732
8,001111100110011100010111111011100111111001000010110100110101,[ 3.6014 -0.3829 11.1759],-153.850827
9,000011100100001110001001111100100001001011001000000011011001,[-24.9619 -13.9223 6.7512],-0.000016


### Fitness medio de la Población Inicial

In [9]:
fitness_medio_inicial = df_poblacion.Fitness.mean()
print(fitness_medio_inicial)

-9.677089496299356


El objetivo es obtener una población (conjunto de soluciones) que tengan un mejor desempeño, es decir, que su fitness medio sea menor que -9,6770. Se realizará un total de 10 corridas (épocas)

### Algoritmo Genético

El mecanismo de selección de los dos padres será aquellos con mejor fitness. Dado que estamos minimizando, se trata de los dos que tengan los menores valores en la función de Ackley 2. El peor individuo de la población será el que tenga mayor valor en la función de Ackley 2.

In [10]:
# Vamos a registrar cada nuevo individuo con un nuevo índice para saber al final cuáles soluciones fueron añadidas y cuáles fueron descartadas
indice_individuo = len(df_poblacion)

# criterio de parada = número total de épocas alcanzadas
for _ in range(0, total_epocas):
    # Ordenamos la población según sus valores de fitness de forma creciente
    df_poblacion = df_poblacion.sort_values(by=['Fitness'])

    # Tomamos los dos mejores individuos, estos son los dos primeros en la lista de población ya que se ha ordenado de forma ascendente según su fitness
    padre1 = df_poblacion.Individuo.values[0]
    padre2 = df_poblacion.Individuo.values[1]

    # Los individuos en el dataframe se encuentran almacenadas como una cadena de caracteres, para poder operar con ellos con el paquete numpy, los convertimos a arreglos utilizando la función "fromiter" que itera sobre la cadena y devuelve el valor de cada caracter como una entrada del arreglo
    padre1 = np.fromiter(padre1, int)
    padre2 = np.fromiter(padre2, int)

    # Realizamos la recombinación para obtener un nuevo individuo
    nuevo_individuo = cruzar(padre1, padre2)

    # Con una probabilidad 'm' mutamos el nuevo cromosoma
    u = np.random.uniform(0, 1) # generamos un valor aleatorio entre 0 y 1 según una distribución uniforme
    if u < m:
        nuevo_individuo = mutar(nuevo_individuo)

    # Creamos un dataframe de una fila con el nuevo individuo con el fin de añadirlo a la población actual
    nuevo_fenotipo = decodificar(nuevo_individuo)
    nuevo_fitness = fitness(nuevo_individuo)
    df_nuevo_individuo = pd.DataFrame([[nuevo_fenotipo, cromosoma_to_string(nuevo_individuo), nuevo_fitness]], columns=['Fenotipo', 'Individuo', 'Fitness'], index=pd.Index([indice_individuo]))

    # Añadimos el nuevo individuo a la población, obteniéndose una nueva época con población de tamaño 21
    df_poblacion = pd.concat([df_poblacion, df_nuevo_individuo])

    # Volvemos a ordenar la población en orden creciente según el fitness
    df_poblacion = df_poblacion.sort_values(by=['Fitness'])

    # Eliminamos el último individuo de la población, el cual es el peor, ya que tiene el mayor valor de fitness
    df_poblacion = df_poblacion[:-1]

    indice_individuo += 1

### Población de la 10º Generación

In [11]:
poblacion_final = df_poblacion.style.set_properties(**{
    'font-size' : '12pt'
})
poblacion_final

,Individuo,Fenotipo,Fitness
8,001111100110011100010111111011100111111001000010110100110101,[ 3.6014 -0.3829 11.1759],-153.850827
22,110000011001100011101000000100001011101000100011011001000111,[-3.6014 -8.7421 24.6058],-123.756678
23,110000011001100011101000000101110100010111011100100110111000,[ -3.6014 8.7267 -24.6058],-114.241813
26,110000011001100011101000000101110100010111011100100110111000,[ -3.6014 8.7267 -24.6058],-114.241813
21,110000011001100011101000000101110100010111011100100110111000,[ -3.6014 8.7267 -24.6058],-114.241813
24,110000011001100011101000000101110100010111011100100110111000,[ -3.6014 8.7267 -24.6058],-114.241813
28,001111100110011100010111111010001011101000100011011001000111,[ 3.6014 -8.7267 24.6058],-107.588888
25,001111100110011100010111111010001011101000100011011001000111,[ 3.6014 -8.7267 24.6058],-107.588888
29,001111100110011100010111111010001011101000100011011001000111,[ 3.6014 -8.7267 24.6058],-107.588888
20,001111100110011100010111111010001011101000100011011001000111,[ 3.6014 -8.7267 24.6058],-107.588888


### Fitness Medio de la Población Final

In [12]:
fitness_medio_final = df_poblacion.Fitness.mean()
print(fitness_medio_final)

-65.6104701770183


### Conclusión

Luego de aplicar el AG al problema de minimización de la función de Ackley 2, hemos obtenido una mejora considerable en el fitness medio de la población, de una media de -9.6770 a una media de -65,6104. Donde la solución con mejor desempeño es 

<center>[ 3,6014 -0,3829 11,1759]</center>

con un valor de la función de Ackley 2 de -153,850827; correspondiente al individuo Nº 8 definido por el siguiente cromosoma:

<center>001111100110011100010111111011100111111001000010110100110101</center>

nacido en la 1era generación del algoritmo.